# Predicting course evaluations with Decision Trees


## 1. CART Algorithm Implementation

### 1.1. Tree data structure and helpers

In [ ]:
class DecisionNode:
    def __init__(self, col=-1, value=None, results=None, tb=None, fb=None):
        self.col = col # attribute on which to split
        self.value = value # value on which to split
        self.results = results # If the node has no children - we store here class labels with their counts
        self.tb = tb  # True branch
        self.fb = fb  # False branch
        
def split(rows, column, value):
    # define split function according to the value type
    split_function = None
    if isinstance(value, int) or isinstance(value, float):
        split_function = lambda row: row[column] >= value
    else:
        split_function = lambda row: row[column] == value

    # Divide the rows into two sets and return them
    set1 = [row for row in rows if split_function(row)]
    set2 = [row for row in rows if not split_function(row)]
    return (set1, set2)

def count_labels(rows):
    label_count = {}
    for row in rows:
        # The class label is in the last column
        label = row[- 1]
        if label not in label_count:
            label_count[label] = 0
        label_count[label] += 1
    return label_count

### 1.2. Node purity scores

In [ ]:
from math import log

def gini_impurity(rows):
    total = len(rows)
    counts = count_labels(rows)
    gini = 0
    for key, val in counts.items():
        p = val / total
        gini += p*p
        
    return (1 - gini)

def entropy(rows):
    total = len(rows)
    counts = count_labels(rows)
    ent = 0.0
    for key,val in counts.items():
        p = val / total
        ent = ent - p * log(p, 2)
    return ent


def variance(rows):
    if len(rows) == 0: return 0
    num_label = [float(row[- 1]) for row in rows]
    mean = sum(num_label) / len(num_label)
    variance = sum([(d - mean) ** 2 for d in num_label]) / len(num_label)
    return variance


### 1.3. Tree induction

In [ ]:
def buildtree(rows, score_func=entropy, min_improvement=0, min_samples=0, max_depth=None, depth=0):
    if len(rows) == 0:
        return DecisionNode()
    # Compute overall score for the entire rows dataset
    current_score = score_func(rows)

    # Set up accumulator variables to track the best split criteria
    best_score = current_score
    best_criteria = None
    best_sets = None
    
    # Total number of features - except the last column where we store the class (target)
    column_count = len(rows[0]) - 1 
    for col in range(0, column_count):
        # Generate the list of unique values in
        # this column to split on them
        column_values = set()
        for row in rows:
            column_values.add(row[col])
            
        # Now try splitting the rows 
        # on each unique value in this column
        for value in column_values:
            (set1, set2) = split(rows, col, value)

            # Evaluate the quality of the split
            # p is the proportion of subset set1 
            p = float(len(set1)) / len(rows)
            split_score = p * score_func(set1) + (1-p) * score_func(set2)
            
            if split_score < best_score and \
                (len(set1) > min_samples and len(set2) > min_samples) and \
                (current_score - split_score) > min_improvement:
                best_score = split_score
                best_criteria = (col, value)
                best_sets = (set1, set2)

    # Create the sub branches
    if (current_score - best_score) > min_improvement and \
        (max_depth is None or depth < max_depth) :
        # print("Splitting on",best_criteria, " 2 sets:", len(best_sets[0]),len(best_sets[1]))
        true_branch = buildtree(best_sets[0], score_func, min_improvement, min_samples, max_depth, depth+1)
        false_branch = buildtree(best_sets[1], score_func, min_improvement, min_samples, max_depth, depth+1)
        return DecisionNode(col=best_criteria[0], value=best_criteria[1],
                            tb=true_branch, fb=false_branch)
    else: # Done splitting - summarize class labels in leaf nodes
        return DecisionNode(results=count_labels(rows))

### 1.4. Tree printing

In [ ]:
def prediction(leaf_labels):
    total = 0
    result = {}
    for label, count in leaf_labels.items():
        total += count
        result[label] = count

    for label, val in result.items():
        result[label] = str(int(result[label]/total * 100))+"%"

    return result

In [ ]:
def print_tree(tree, current_branch, attributes=None,  indent='', leaf_funct=prediction):
    # Is this a leaf node?
    if tree.results != None:
        print(indent + current_branch + str(leaf_funct(tree.results)))
    else:
        # Print the split question
        split_col = str(tree.col)
        if attributes is not None:
            split_col = attributes[tree.col]
        split_val = str(tree.value)
        if type(tree.value) == int or type(tree.value) == float:
            split_val = ">=" + str(tree.value)
        print(indent + current_branch + split_col + ': ' + split_val + '? ')

        # Print the branches
        indent = indent + '  '
        print_tree(tree.tb, 'T->', attributes, indent)
        print_tree(tree.fb, 'F->', attributes, indent)

### 1.5. Classification

In [ ]:
def classify(observation, tree):
    if tree.results != None:
        return prediction(tree.results)
    else:
        v = observation[tree.col]
        branch = None
        if isinstance(v, int) or isinstance(v, float):
            if v >= tree.value:
                branch = tree.tb
            else:
                branch = tree.fb
        else:
            if v == tree.value:
                branch = tree.tb
            else:
                branch = tree.fb
        return classify(observation, branch)


# Classify an observation with missing data
def mdclassify(observation, tree):
    if tree.results != None:
        return prediction(tree.results)
    else:
        v = observation[tree.col]
        if v == None:
            tr, fr = mdclassify(observation, tree.tb), mdclassify(observation, tree.fb)
            tcount = sum(tr.values())
            fcount = sum(fr.values())
            tw = float(tcount) / (tcount + fcount)
            fw = float(fcount) / (tcount + fcount)
            result = {}
            for k, v in tr.items(): result[k] = v * tw
            for k, v in fr.items(): result[k] = v * fw
            return result
        else:
            if isinstance(v, int) or isinstance(v, float):
                if v >= tree.value:
                    branch = tree.tb
                else:
                    branch = tree.fb
            else:
                if v == tree.value:
                    branch = tree.tb
                else:
                    branch = tree.fb
            return mdclassify(observation, branch)

def max_depth(tree):
    if tree.results != None:
        return 0
    else:
        # Compute the depth of each subtree
        tDepth = max_depth(tree.tb)
        fDepth = max_depth(tree.fb)

        # Use the larger one
        if (tDepth > fDepth):
            return tDepth + 1
        else:
            return fDepth + 1

## 2. Dataset to numeric

The dataset for this lab contains about 450 anonymized student evaluations collected at the University of Texas at Austin, and used in the following publication: __"Beauty in the Classroom: Instructors' Pulchritude and Putative Pedagogical Productivity"__. You can learn how the data was collected and the meaning of various data attributes following this [link](https://chance.amstat.org/2013/04/looking-good/).

We use a subset of attributes: 'rank', 'ethnicity', 'gender', 'language', 'age', 'class_size', 'class_level',  'avg_beauty_score',  and 'professor_evaluation_score' and try to predict course evaluation score. 

This smaller dataset can be downloaded from [here](https://drive.google.com/file/d/18wV59AYCVCqL1BIDBYLpL73emp7h8d0T/view?usp=sharing). Download the dataset into your data directory, and update the path to your file in the cell below.

In [ ]:
data_file = "../data_sets/StudentEvaluations.csv"

We read the dataset into a data frame _df_. First, we will make all attributes numeric, and then we will convert all attributes to categorical labels. The _df_ points to the original dataset. We are going to create two copies: _df_num_ and _df_cat_.

In [ ]:
import pandas as pd
df = pd.read_csv(data_file)

In [ ]:
print(df.columns)

In [ ]:
print(len(df))

In [ ]:
print(df[:7])

We see that not all attributes are numeric. Let's make them all numeric by using the pandas _replace_ method (see [this post](https://www.geeksforgeeks.org/how-to-convert-categorical-variable-to-numeric-in-pandas/)), which replaces the original values with the predefined set.

In [ ]:
# First, create a copy of the original data frame
df_num = df.copy()

We need to replace 'rank', 'ethnicity', 'gender', 'language', and 'cls_level' columns.
Let's find out what are the values in each column - to know what to replace.

In [ ]:
# Rank
df_num['rank'].unique()

In [ ]:
# Ethnicity
df_num['ethnicity'].unique()

In [ ]:
# Gender
df_num['gender'].unique()

In [ ]:
# Language
df_num['language'].unique()

In [ ]:
# Class level
df_num['cls_level'].unique()

Assigning a numeric code to each category.

In [ ]:
df_num['rank'].replace(['tenure track', 'tenured', 'teaching'], [1, 2, 3], inplace=True)
df_num['ethnicity'].replace(['minority', 'not minority'], [1, 0], inplace=True)
df_num['gender'].replace(['female', 'male'], [1, 2], inplace=True)
df_num['language'].replace(['english', 'non-english'], [1, 0], inplace=True)
df_num['cls_level'].replace(['upper', 'lower'], [2,1], inplace=True)

Make the class label an integer by rounding course evaluation scores.

In [ ]:
df_num['course_eval'] = df_num['course_eval'].round(0).astype(int)

df_num['course_eval'].dtype

And check what it looks like now:

In [ ]:
df_num[:7]

Let's explore if there is any correlation between course score and any other attributes in the dataset.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


corr = df_num.corr()
sns.heatmap(corr)

## 3. Decision Tree: numeric dataset
### 3.1. Using custom ID3 algorithm

In [ ]:
data_rows = df_num.to_numpy().tolist()

In [ ]:
columns_list = df_num.columns.to_numpy().tolist()

This would be a regression tree, as we are trying to predict a numeric score. We have to use __variance__ as a measure of node purity.

In [ ]:
tree = buildtree(data_rows, score_func=variance, min_improvement=0, min_samples=0)

In [ ]:
print_tree(tree, '', columns_list)

### 3.2. Using sklearn library
The decision tree algorithm in sklearn library is not implemented very well. It requires all the attributes to be numeric - while decision trees work best with the categorical attributes. Nevertheless, we will try to run it and compare the results.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import tree

The class label has to be binary. So we are going to replace the 'course_eval' column with 3 bins, corresponding to the score:

Divide dataset into features and class label.

In [ ]:
X = df_num.loc[:, df_num.columns != 'course_eval']
print(X.columns)

Y = df_num.loc[:, df_num.columns == 'course_eval']
print(Y.columns)

Split dataset intro training and testing.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)

Different parameters can be specified to build the model:

`model = tree.DecisionTreeClassifier(
        criterion='entropy', 
        max_depth=None, 
        min_samples_split=2, 
        min_samples_leaf=1, 
        max_features=None, 
        random_state=None, 
        min_density=None, 
        compute_importances=None, 
        max_leaf_nodes=None)`

In [ ]:
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth = 7)
model.fit(X_train,Y_train)

In [ ]:
columns_list = df_num.columns.to_numpy().tolist()
from sklearn.tree import export_text
r = export_text(model, feature_names=columns_list[:-1])
print(r)

## 4. Categorical dataset

We are going to replace all the numeric columns in _df_ by categorical labels, using [Pandas _cut_ method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html).

First we create a copy of the original dataset.

In [ ]:
df_cat = df.copy()

In [ ]:
print(df_cat.columns)
print(df_cat[:5])

We need to bin columns 'age', 'cls_students', 'bty_avg', 'prof_eval', and 'course_eval'. Everything else is already categorical.

First thing is to determine the domain for each numeric column.

In [ ]:
# Age
age_column = df_cat["age"]
print("From:", age_column.min(), "to", age_column.max())

In [ ]:
# Beauty score
bty_ave_column = df_cat["bty_avg"]
print("From:", bty_ave_column.min(), "to", bty_ave_column.max())

In [ ]:
# Class size
num_students_column = df_cat["cls_students"]
print("From:", num_students_column.min(), "to", num_students_column.max())

In [ ]:
# Professor score
prof_eval_column = df_cat["prof_eval"]
print("From:", prof_eval_column.min(), "to", prof_eval_column.max())

In [ ]:
# Course score (class label)
course_eval_column = df_cat["course_eval"]
print("From:", course_eval_column.min(), "to", course_eval_column.max())

In [ ]:
import numpy as np

# Age to bins
bins = [20, 35, 50, 65, np.inf]
names = [ '20-35', '35-50',  '50-65', '65+']

df_cat['age'] = pd.cut(df_cat['age'], bins, labels=names)

# Class size to bins
bins = [0, 15, 30, 50, 100, np.inf]
names = [ '<15', '15-30',  '30-50', '50-100', '100+']

df_cat['cls_students'] = pd.cut(df_cat['cls_students'], bins, labels=names)

# Beauty average to bins
bins = [0, 3, 6, 9, np.inf]
names = [ '<3', '3-6',  '6-9', '9+']

df_cat['bty_avg'] = pd.cut(df_cat['bty_avg'], bins, labels=names)

# Professor score to bins
bins = [0, 2, 3, 4, 4.5, np.inf]
names = [ '<2', '2-3',  '3-4', '4-4.5', '4.5+']

df_cat['prof_eval'] = pd.cut(df_cat['prof_eval'], bins, labels=names)

# Course score to class label
bins = [0, 2, 3, 4, 4.5, np.inf]
names = [ 'bad', 'fair',  'average', 'good', 'excellent']

df_cat['course_eval'] = pd.cut(df_cat['course_eval'], bins, labels=names)


In [ ]:
print(df_cat.columns)
print(df_cat[:5])

## 5. Decision Tree: categorical dataset

In [ ]:
data_rows = df_cat.to_numpy().tolist()

In [ ]:
columns_list = df_cat.columns.to_numpy().tolist()

Now this is a decision tree (classification), we use entropy as a measure of leaf purity.

In [ ]:
tree = buildtree(data_rows, score_func=entropy, min_improvement=0, min_samples=0)

In [ ]:
print_tree(tree, '', columns_list)

This is the best we could do for this dataset.

## 6. [TASK 1] Using learned tree for classification

Recall at least 3 different courses you have taken, and record the values of the attributes, and the course score that you have given to the course. Run the predict function to see if the decision tree correctly predicts your score.

## 7. [TASK 2] Important factors
What are the most important attributes in separating good course evaluations from the bad ones? Support your answer by analyzing the tree levels and the leaf outcomes.

## 8. [TASK 3] Rules
Extract from the tree at least 3 rules that you find most reliable. Did any of these rules surprise you?

## 9. [BONUS] Web visualization
The visualizations of the tree can be made much more expressive with the _d3.js_ visualization library. Sample  _public_html_ folder containing all the required html, css, and javascript files, including the library itself, is provided. 

In order to create this visualization, all you need is a json file which represents the nodes of the final decision tree. Learn the required JSON format, and implement this conversion in a separate file __dtree_to_json.py__. Submit your file with the lab. 

Copy the resulting json file into the data directory in the _public_html_ folder, and update the name of the json file in the JavaScript code on line 39 in _index.html_. 

Note that Chrome would not allow you to run the visualization locally - so if you want to present it to the world you should use a web server. 


Copyright &copy; 2020 Marina Barsky. All rights reserved.